In [1]:
import numpy as np
import scipy as sc
import operators.plaqOps as plaq_states

In [2]:
def count_bits(n):
    """Returns array where result[i] = number of 1s in binary(i) for i in [0, n-1]"""
    result = [0] * n
    for i in range(1, n):
        result[i] = result[i >> 1] + (i & 1)
    return result

def get_pos(nx, ny, x, y):
    return nx*y+x

def get_bits(num, i, j, n):
    """Returns array [bit_i, bit_j] where i and j are 0-indexed from the left.
    n is the total binary length."""
    pos_i = n - 1 - i
    pos_j = n - 1 - j
    return [(num >> pos_i) & 1, (num >> pos_j) & 1]

def flip_bits(num, i, j, n):
    """Flip the ith and jth bits (0-indexed from left).
    n is the total binary length. Returns tuple (modified_number, bits_were_same)."""
    pos_i = n - 1 - i
    pos_j = n - 1 - j
    # Get the bits before flipping
    bit_i = (num >> pos_i) & 1
    bit_j = (num >> pos_j) & 1
    bits_were_same = (bit_i == bit_j)
    # Flip the bits
    num ^= (1 << pos_i)
    num ^= (1 << pos_j)
    return num, bits_were_same



In [3]:
def H(nx,ny,g2):
    oL, Lsq, ImTrP = plaq_states._getPlaqStateOps(0.5, g2)

    vol = nx*ny
    dim = 2**(vol)
    x_index = []
    y_index = []
    data = []

    local = g2/2 * 4* Lsq + 1/2/g2*ImTrP
    offset = np.real(local[0,0])
    gap = np.real(local[1,1]-local[0,0])

    x_index += list(np.arange(dim))
    y_index += list(np.arange(dim))
    data = data+list(np.array(count_bits(dim))*gap+vol*offset)

    same = np.real(np.kron(oL, oL)[3,0])
    diff = np.real(np.kron(oL, oL)[2,1])

    for x in range(nx-1):
        for y in range(ny-1):
            s = get_pos(nx, ny, x, y)
            n1 = get_pos(nx, ny, x+1, y)
            n2 = get_pos(nx, ny, x, y+1)
            for j in range(dim):
                x_index += [j,j]
                k1, l1 = flip_bits(j, s, n1, vol)
                k2, l2 = flip_bits(j, s, n2, vol)
                y_index += [k1,k2]
                data += [diff+(same-diff)*l1,diff+(same-diff)*l2]

    for x in range(nx-1):
        s = get_pos(nx, ny, x, ny-1)
        n1 = get_pos(nx, ny, x+1, ny-1)
        for j in range(dim):
            x_index += [j]
            k1, l1 = flip_bits(j, s, n1, vol)
            y_index += [k1]
            data += [diff+(same-diff)*l1]


    for y in range(ny-1):
        s = get_pos(nx, ny, x, y)
        n2 = get_pos(nx, ny, x, y+1)
        for j in range(dim):
            x_index += [j]
            k2, l2 = flip_bits(j, s, n2, vol)
            y_index += [k2]
            data += [diff+(same-diff)*l2]


    # add two edge cases
    return sc.sparse.coo_matrix((data,(x_index,y_index)), dtype=np.float64)


In [4]:
def diagonalize_hamiltonian(nx, ny, g2, save_to_disk=True, save_dir='diagonalization_results'):
    """
    Fully diagonalize the Hamiltonian H(nx, ny, g2).
    
    Parameters:
    -----------
    nx, ny : int
        Lattice dimensions
    g2 : float
        Coupling parameter
    save_to_disk : bool
        If True, saves eigenvalues and eigenvectors to disk
    save_dir : str
        Directory to save results
        
    Returns:
    --------
    eigenvalues : ndarray
        Array of eigenvalues (sorted)
    eigenvectors : ndarray
        Matrix of eigenvectors (columns are eigenvectors)
    """
    import os
    
    print(f"Constructing Hamiltonian for nx={nx}, ny={ny}, g2={g2}...")
    H_sparse = H(nx, ny, g2)
    
    vol = nx * ny
    dim = 2**vol
    print(f"Matrix dimension: {dim}x{dim}")
    
    # Estimate memory usage (real symmetric matrix)
    memory_gb = dim * dim * 8 / (1024**3)  # float64 = 8 bytes
    print(f"Estimated memory for dense matrix: {memory_gb:.2f} GB")
    
    if memory_gb > 8:
        print("Warning: Large memory footprint. Converting to dense and diagonalizing...")
    
    # Convert to dense for full diagonalization
    print("Converting sparse to dense...")
    H_dense = H_sparse.toarray()
    
    # Diagonalize using hermitian eigenvalue solver (works for real symmetric too)
    print("Diagonalizing...")
    eigenvalues, eigenvectors = np.linalg.eigh(H_dense)
    
    print(f"Diagonalization complete. Found {len(eigenvalues)} eigenvalues.")
    print(f"Energy range: [{eigenvalues[0]:.6f}, {eigenvalues[-1]:.6f}]")
    
    if save_to_disk:
        # Create filename
        filename_base = f"nx{nx}_ny{ny}_g2{g2:.4f}".replace('.', 'p')
        
        os.makedirs(save_dir, exist_ok=True)
        
        eigenval_file = os.path.join(save_dir, f"{filename_base}_eigenvalues.npy")
        eigenvec_file = os.path.join(save_dir, f"{filename_base}_eigenvectors.npy")
        
        print(f"Saving eigenvalues to {eigenval_file}...")
        np.save(eigenval_file, eigenvalues)
        
        print(f"Saving eigenvectors to {eigenvec_file}...")
        np.save(eigenvec_file, eigenvectors)
        
        print("Results saved successfully.")
    
    return eigenvalues, eigenvectors


In [5]:
def diagonalize_by_parity(nx, ny, g2, save_to_disk=True, save_dir='diagonalization_results'):
    """
    Fully diagonalize the Hamiltonian by exploiting parity symmetry (even/odd bit count).
    This reduces memory by a factor of ~4 since we diagonalize two blocks of size ~dim/2 each.
    
    Parameters:
    -----------
    nx, ny : int
        Lattice dimensions
    g2 : float
        Coupling parameter
    save_to_disk : bool
        If True, saves eigenvalues and eigenvectors to disk
    save_dir : str
        Directory to save results
        
    Returns:
    --------
    eigenvalues : ndarray
        Array of all eigenvalues (sorted)
    parity_info : dict
        Information about each parity sector
    """
    import os
    
    print(f"Constructing Hamiltonian for nx={nx}, ny={ny}, g2={g2}...")
    H_sparse = H(nx, ny, g2).tocsr()
    
    vol = nx * ny
    dim = 2**vol
    print(f"Full matrix dimension: {dim}x{dim}")
    
    # Group states by parity (even/odd bit count)
    print("Organizing states into parity sectors...")
    bit_counts = count_bits(dim)
    even_states = [i for i in range(dim) if bit_counts[i] % 2 == 0]
    odd_states = [i for i in range(dim) if bit_counts[i] % 2 == 1]
    
    even_size = len(even_states)
    odd_size = len(odd_states)
    
    print(f"  Even parity sector: {even_size} states ({100*even_size/dim:.1f}%)")
    print(f"  Odd parity sector:  {odd_size} states ({100*odd_size/dim:.1f}%)")
    
    even_mem_gb = even_size * even_size * 8 / (1024**3)
    odd_mem_gb = odd_size * odd_size * 8 / (1024**3)
    print(f"  Even sector memory: {even_mem_gb:.2f} GB")
    print(f"  Odd sector memory:  {odd_mem_gb:.2f} GB")
    print(f"  Total: {even_mem_gb + odd_mem_gb:.2f} GB (vs {dim*dim*8/(1024**3):.2f} GB for full matrix)")
    
    all_eigenvalues = []
    parity_info = {}
    
    if save_to_disk:
        filename_base = f"nx{nx}_ny{ny}_g2{g2:.4f}".replace('.', 'p')
        os.makedirs(save_dir, exist_ok=True)
    
    # Diagonalize even parity sector
    if even_size > 0:
        print(f"\n=== Diagonalizing EVEN parity sector ({even_size} states) ===")
        print("  Extracting block submatrix...")
        even_block = H_sparse[np.ix_(even_states, even_states)].toarray()
        
        print("  Diagonalizing...")
        even_eigenvalues, even_eigenvectors = np.linalg.eigh(even_block)
        
        print(f"  Complete! Energy range: [{even_eigenvalues[0]:.6f}, {even_eigenvalues[-1]:.6f}]")
        
        all_eigenvalues.append(even_eigenvalues)
        parity_info['even'] = {
            'size': even_size,
            'states': even_states,
            'energy_range': (even_eigenvalues[0], even_eigenvalues[-1])
        }
        
        if save_to_disk:
            even_file_base = f"{filename_base}_even"
            np.save(os.path.join(save_dir, f"{even_file_base}_eigenvalues.npy"), even_eigenvalues)
            np.save(os.path.join(save_dir, f"{even_file_base}_eigenvectors.npy"), even_eigenvectors)
            np.save(os.path.join(save_dir, f"{even_file_base}_states.npy"), np.array(even_states))
            print(f"  Saved to {save_dir}")
        
        del even_block, even_eigenvectors  # Free memory
    
    # Diagonalize odd parity sector
    if odd_size > 0:
        print(f"\n=== Diagonalizing ODD parity sector ({odd_size} states) ===")
        print("  Extracting block submatrix...")
        odd_block = H_sparse[np.ix_(odd_states, odd_states)].toarray()
        
        print("  Diagonalizing...")
        odd_eigenvalues, odd_eigenvectors = np.linalg.eigh(odd_block)
        
        print(f"  Complete! Energy range: [{odd_eigenvalues[0]:.6f}, {odd_eigenvalues[-1]:.6f}]")
        
        all_eigenvalues.append(odd_eigenvalues)
        parity_info['odd'] = {
            'size': odd_size,
            'states': odd_states,
            'energy_range': (odd_eigenvalues[0], odd_eigenvalues[-1])
        }
        
        if save_to_disk:
            odd_file_base = f"{filename_base}_odd"
            np.save(os.path.join(save_dir, f"{odd_file_base}_eigenvalues.npy"), odd_eigenvalues)
            np.save(os.path.join(save_dir, f"{odd_file_base}_eigenvectors.npy"), odd_eigenvectors)
            np.save(os.path.join(save_dir, f"{odd_file_base}_states.npy"), np.array(odd_states))
            print(f"  Saved to {save_dir}")
        
        del odd_block, odd_eigenvectors  # Free memory
    
    # Combine and sort all eigenvalues
    all_eigenvalues = np.concatenate(all_eigenvalues)
    all_eigenvalues.sort()
    
    print(f"\n=== Full Diagonalization Complete ===")
    print(f"Total eigenvalues: {len(all_eigenvalues)}")
    print(f"Ground state energy: {all_eigenvalues[0]:.6f}")
    print(f"Max energy: {all_eigenvalues[-1]:.6f}")
    
    if save_to_disk:
        all_eigenval_file = os.path.join(save_dir, f"{filename_base}_all_eigenvalues.npy")
        np.save(all_eigenval_file, all_eigenvalues)
        print(f"All eigenvalues saved to {all_eigenval_file}")
    
    return all_eigenvalues, parity_info


In [ ]:
diagonalize_by_parity(4, 4, 0.1, save_to_disk=True, save_dir='diagonalization_results')

Constructing Hamiltonian for nx=4, ny=4, g2=0.1...
Full matrix dimension: 65536x65536
Organizing states into parity sectors...
  Even parity sector: 32768 states (50.0%)
  Odd parity sector:  32768 states (50.0%)
  Even sector memory: 8.00 GB
  Odd sector memory:  8.00 GB
  Total: 16.00 GB (vs 32.00 GB for full matrix)

=== Diagonalizing EVEN parity sector (32768 states) ===
  Extracting block submatrix...
